In [1]:
import json
import os
from tqdm import tqdm
import pandas as pd
import re

In [2]:
def ticker_adder(pos_list):
    prev_ticker = "NONE"
    for group in pos_list:
        if not group[0]:
            group[0] = prev_ticker
        else:
            prev_ticker = group[0]
    return pos_list

In [12]:
def regex_pos(post_text):
    if pd.isna(post_text): return 0
    x = []
    p1 = re.compile('(\$?[A-Z]{1,4})?\s+\$?(?:(\d+(?:\.\d\d?)?)\s*([CcPp])\w{0,4}\s+(\d\d?\/\d\d?(?:\/\d{2,4})?))')
    p2 = re.compile('(\$?[A-Z]{1,4})?\s+(\d\d?\/\d\d?(?:\/\d{2,4})?)\s+\$?(\d+(?:\.\d\d?)?)\s*([CcPp])\w{0,4}')
    p3 = re.compile('(\d\d?\/\d\d?(?:\/\d{2,4})?)\s+(\$?[A-Z]{1,4})\s+\$?(\d+(?:\.\d\d?)?)\s*([CcPp])\w{0,4}')
    p4 = re.compile('(\$?[A-Z]{1,4})?\s+(\d{1,2}\s?[A-Z]{1,3})\s+\$?(\d+(?:\.\d\d?)?)\s*([CcPp])\w{0,4}')
    
    # (\$?[A-Z]{1,4})\s+(?:\$?(\d+(?:.\d\d)?)\s*([CcPp])\w{0,4}\s+(\d\d?\/\d\d?(?:\/\d{2,4})?)+[^A-Z]*)
    
    x1 = re.findall(p1, post_text)
    x2 = re.findall(p2, post_text)
    x3 = re.findall(p3, post_text)
    x4 = re.findall(p4, post_text) # diving into alpha-numeric dates like 20 NOV
    
    #dictating ticker-strike-c/p-date as standard. x1 stays, the rest must be edited 
    if x1:
        x1_e = []
        for match in x1:
            ordered = [match[0], match[1], match[2], match[3]]
            x1_e.append(ordered)
        x.extend(x1_e)
    if x2: #ticker-date-strike-c/p
        x2_e = []
        for match in x2:
            ordered = [match[0],match[2],match[3],match[1]]
            x2_e.append(ordered)
        x.extend(x2_e)
    if x3: #date-ticker-strike-c/p
        x3_e = []
        for match in x3:
            ordered = [match[1],match[2],match[3],match[0]]
            x3_e.append(ordered)
        x.extend(x3_e)
    if x4: # ticker-date(alphanumeric)-stike-c/p
        x4_e = []
        for match in x4:
            ordered = [match[0],match[2],match[3],match[1],'x4']
            x4_e.append(ordered)
        x.extend(x4_e)
    if x:
        x = ticker_adder(x)
        return x 
    else:
        return 0

In [ ]:
def year_adder(row):
    post_date = row['created_utc']
    body = row['regexed_body']
    title = row['title']
    
    # get the created year, compare to expiry
    # if expiry contains year, confirm format YYYY
    # else record YYYY based on post date (i.e post of 1/2020 will not have 12/2019 in expiry)

In [13]:
## maybe create a number of posts function for a new col
## need a expiry function to get expiry out of position
## stock at open and close (where close applicable)

In [14]:
good_post_csv = pd.read_csv('all_good_posts.csv')
all_post_csv = pd.read_csv('all_posts.csv')

In [15]:
good_post_csv['regexed_body'] = good_post_csv['selftext'].apply(regex_pos)
good_post_csv['regexed_title'] = good_post_csv['title'].apply(regex_pos)

In [16]:
good_post_csv[good_post_csv['regexed_body'] != 0]

,Unnamed: 0,author,created_utc,title,link_flair_text,selftext,id,full_link,regexed_body,regexed_title
6,6,angryrantingdude,1569872663,Should I cut my loss or turn my puts into a st...,Options,So I bought $T $37.5p 10/4 expiration. But ob...,dblal0,https://www.reddit.com/r/wallstreetbets/commen...,"[[$T, 37.5, p, 10/4]]",0
159,159,lost_civilizations,1585016402,Hotels and Airline bailouts,Fundamentals,Wake up people - The only reason these compani...,fo0vym,https://www.reddit.com/r/wallstreetbets/commen...,"[[HLT, 40, P, 5/1]]",0
161,161,CodfishCannon,1585017606,AMZN Call Spread Bug,YOLO,Found a fun little Robinhood glitch: more valu...,fo13kb,https://www.reddit.com/r/wallstreetbets/commen...,"[[HDB, 35, p, 4/17], [EWP, 19, p, 4/17], [BLK,...",0
210,210,jlk1994-,1585034103,Was I misdiagnosed?,Discussion,"I got diagnosed with Asperger, like an actual ...",fo44sm,https://www.reddit.com/r/wallstreetbets/commen...,"[[VTI, 70, p, 4/17]]",0
215,215,therealrogi,1585034827,Accumulation phase ACTIVATE,Fundamentals,"Feel familiar?\n\n""But how does an accumulatio...",fo4a5p,https://www.reddit.com/r/wallstreetbets/commen...,"[[SPY, 200, p, 4/13], [SPY, 205, p, 4/15], [SP...",0
...,...,...,...,...,...,...,...,...,...,...
117537,117537,DisputablyGreen,1604935956,"If you liked Burry on GME, you're gonna love h...",DD,"Alright, idiots, here’s DISCA.\n\nBurry owns ...",jr5ohd,https://www.reddit.com/r/wallstreetbets/commen...,"[[NONE, 22.5, c, 12/18]]",0
117542,117542,sjtomcat,1604936049,PLTR Gang how we feeling,Discussion,How we all feeling after today? Seems like som...,jr5png,https://www.reddit.com/r/wallstreetbets/commen...,"[[NONE, 16, c, 11/13]]",0
117549,117549,yeeterboi1234567,1604937591,Who else went all in on RKT today?,YOLO,Went all in on RKT 11/13 25c lol. Who else?,jr68pb,https://www.reddit.com/r/wallstreetbets/commen...,"[[RKT, 25, c, 11/13]]",0
117570,117570,Babyyodaenergy,1604939196,Nio Calls DEC,Discussion,How are we feeling about December NIO calls \...,jr6slg,https://www.reddit.com/r/wallstreetbets/commen...,"[[NONE, 42, C, 12/11], [NONE, 50, C, 1/15/21]]",0


In [17]:
good_post_csv[good_post_csv['regexed_title'] != 0]

,Unnamed: 0,author,created_utc,title,link_flair_text,selftext,id,full_link,regexed_body,regexed_title
136,136,clmohn,1585011346,COVID-19 was just something printing me tendie...,Discussion,[removed],fnzzcm,https://www.reddit.com/r/wallstreetbets/commen...,0,"[[NONE, 200, P, 5/15]]"
306,306,B1G_Peter,1585043577,Universities are starting to bring back studen...,Discussion,[https://www.newsadvance.com/news/local/thousa...,fo6e2g,https://www.reddit.com/r/wallstreetbets/commen...,"[[SPY, 222, p, 3/30]]","[[SPY, 50, p, 4/17]]"
528,528,llllllllhhhhhhhhh,1585059667,Walmart 120c 4/17,Discussion,So I got this bad boy this morning. I don’t an...,fob8vs,https://www.reddit.com/r/wallstreetbets/commen...,0,"[[NONE, 120, c, 4/17]]"
743,743,cleveland-216,1585073810,GE 7c 3/27,Options,Going all in on GE futures at 7c 3/27 on a qui...,fofhbq,https://www.reddit.com/r/wallstreetbets/commen...,"[[NONE, 7, c, 3/27]]","[[GE, 7, c, 3/27]]"
750,750,futuretrollshark,1585074464,Buy MTCH 60c 4/17,DD,Tinder had a 15% UPTICK in downloads. Wanna k...,fofnzc,https://www.reddit.com/r/wallstreetbets/commen...,"[[NONE, 60, c, 4/17]]","[[MTCH, 60, c, 4/17]]"
...,...,...,...,...,...,...,...,...,...,...
117438,117438,maR-08,1604926020,ICLN $26C 1/15/20 To the MOON 🚀🚀,Discussion,NaN,jr2c1x,https://www.reddit.com/r/wallstreetbets/commen...,0,"[[ICLN, 26, C, 1/15/20]]"
117439,117439,maR-08,1604926092,ICLN $26 Call 1/15/21 🔥🔥🚀🚀,Discussion,NaN,jr2cvt,https://www.reddit.com/r/wallstreetbets/commen...,0,"[[ICLN, 26, C, 1/15/21]]"
117467,117467,Dan_inKuwait,1604928694,BNTX 105c 11/20 - diamond hands or eat the mar...,Discussion,"Hey, king retard here.\n\n&amp;#x200B;\n\nIn a...",jr386y,https://www.reddit.com/r/wallstreetbets/commen...,"[[$BNTX, 105, c, 11/20]]","[[BNTX, 105, c, 11/20]]"
117557,117557,TheGuyWhoCodes,1604938238,NIO 43c 11/13 How we feeling?,Discussion,NaN,jr6gtz,https://www.reddit.com/r/wallstreetbets/commen...,0,"[[NIO, 43, c, 11/13]]"


In [7]:
#good_post_csv.to_csv('output1.csv')

In [18]:
good_post_csv.iloc[117573]['title']

'PLTR gang'

In [19]:
good_post_csv.iloc[117573]['selftext']

'Y’all holding long term? Where do you see it heading?\n\nPositions: \n10c 5/21/20\n67 shares'

In [22]:
good_post_csv.iloc[117573]['regexed_body']

[['NONE', '10', 'c', '5/21/20']]